<a href="https://colab.research.google.com/github/naveen4006/Fmml_labs/blob/main/Copy_of_Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
Pprint('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

1)Yes, averaging validation accuracy across multiple splits, typically in a technique called cross-validation, can provide more consistent and reliable results compared to a single validation split. Cross-validation helps reduce the impact of randomness and variations in data splitting, leading to a better estimate of a model's performance on unseen data. It provides a more robust assessment of a model's generalization capabilities. Common cross-validation methods include k-fold cross-validation and stratified cross-validation.

2)
Yes, averaging validation accuracy across multiple splits, typically in a technique called cross-validation, can provide more consistent and reliable results compared to a single validation split. Cross-validation helps reduce the impact of randomness and variations in data splitting, leading to a better estimate of a model's performance on unseen data. It provides a more robust assessment of a model's generalization capabilities. Common cross-validation methods include k-fold cross-validation and stratified cross-validation.




User
Does it give more accurate estimate of test accuracy?

ChatGPT
Cross-validation, including methods like k-fold cross-validation and stratified cross-validation, can provide a more accurate estimate of a model's generalization performance compared to a single validation split. This is especially important when you have a limited amount of data.

By averaging the performance across multiple validation sets (folds), you get a more comprehensive view of how well your model is likely to perform on unseen data. It helps mitigate the risk of overfitting to a particular validation set or getting overly optimistic or pessimistic estimates from a single split. However, it's important to note that cross-validation still estimates the model's performance on validation data, not on a true "test" data set that is completely unseen during training.

For a true estimate of test accuracy, you should reserve a separate, untouched test dataset that the model has never seen during training or validation. This final evaluation provides the most accurate estimate of how well your model is expected to perform in a real-world scenario. Cross-validation helps you tune your model and understand its generalization capabilities, but the final evaluation on a dedicated test set is crucial for assessing its real-world performance.

3)
Yes, averaging validation accuracy across multiple splits, typically in a technique called cross-validation, can provide more consistent and reliable results compared to a single validation split. Cross-validation helps reduce the impact of randomness and variations in data splitting, leading to a better estimate of a model's performance on unseen data. It provides a more robust assessment of a model's generalization capabilities. Common cross-validation methods include k-fold cross-validation and stratified cross-validation.




User
Does it give more accurate estimate of test accuracy?


Cross-validation, including methods like k-fold cross-validation and stratified cross-validation, can provide a more accurate estimate of a model's generalization performance compared to a single validation split. This is especially important when you have a limited amount of data.

By averaging the performance across multiple validation sets (folds), you get a more comprehensive view of how well your model is likely to perform on unseen data. It helps mitigate the risk of overfitting to a particular validation set or getting overly optimistic or pessimistic estimates from a single split. However, it's important to note that cross-validation still estimates the model's performance on validation data, not on a true "test" data set that is completely unseen during training.

For a true estimate of test accuracy, you should reserve a separate, untouched test dataset that the model has never seen during training or validation. This final evaluation provides the most accurate estimate of how well your model is expected to perform in a real-world scenario. Cross-validation helps you tune your model and understand its generalization capabilities, but the final evaluation on a dedicated test set is crucial for assessing its real-world performance.




User
What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

The number of iterations or epochs in training a machine learning model can have an impact on the estimate of its performance, but it's not necessarily the case that higher iterations always lead to a better estimate. The relationship between the number of iterations and the estimate of performance depends on various factors:

Underfitting and Overfitting: If you have too few iterations, your model may underfit the training data and have poor generalization. Increasing the number of iterations can help the model fit the data better, reducing underfitting. However, if you increase iterations excessively, the model might start to overfit the training data, which can lead to poor generalization on unseen data.

Learning Rate: The learning rate, which controls the size of parameter updates during training, can interact with the number of iterations. Higher learning rates might require fewer iterations to converge, but they can also lead to instability or overshooting. Smaller learning rates might necessitate more iterations for convergence.

Early Stopping: Many training procedures use techniques like early stopping to prevent overfitting. In this case, training stops when the model's performance on a validation set starts to degrade, which means the number of iterations is dynamically determined.

Data Complexity: The complexity of your dataset and the model architecture also play a role. Complex models on complex datasets might require more iterations to reach convergence.

Computational Resources: Training for a higher number of iterations can be computationally expensive. You might need to balance computational constraints with the need for more iterations.

In practice, it's common to use techniques like cross-validation or validation sets to find an appropriate number of iterations that balances underfitting and overfitting. You typically monitor the model's performance on validation data and choose the number of iterations that yields the best generalization without overfitting.

So, while increasing the number of iterations can help improve model performance up to a point, there's no guarantee that more iterations always lead to a better estimate, as it depends on several factors, including those mentioned above. The key is to find the right balance during model training to achieve the best generalization performance.

4)
Yes, averaging validation accuracy across multiple splits, typically in a technique called cross-validation, can provide more consistent and reliable results compared to a single validation split. Cross-validation helps reduce the impact of randomness and variations in data splitting, leading to a better estimate of a model's performance on unseen data. It provides a more robust assessment of a model's generalization capabilities. Common cross-validation methods include k-fold cross-validation and stratified cross-validation.




User
Does it give more accurate estimate of test accuracy?

ChatGPT
Cross-validation, including methods like k-fold cross-validation and stratified cross-validation, can provide a more accurate estimate of a model's generalization performance compared to a single validation split. This is especially important when you have a limited amount of data.

By averaging the performance across multiple validation sets (folds), you get a more comprehensive view of how well your model is likely to perform on unseen data. It helps mitigate the risk of overfitting to a particular validation set or getting overly optimistic or pessimistic estimates from a single split. However, it's important to note that cross-validation still estimates the model's performance on validation data, not on a true "test" data set that is completely unseen during training.

For a true estimate of test accuracy, you should reserve a separate, untouched test dataset that the model has never seen during training or validation. This final evaluation provides the most accurate estimate of how well your model is expected to perform in a real-world scenario. Cross-validation helps you tune your model and understand its generalization capabilities, but the final evaluation on a dedicated test set is crucial for assessing its real-world performance.




User
What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

ChatGPT
The number of iterations or epochs in training a machine learning model can have an impact on the estimate of its performance, but it's not necessarily the case that higher iterations always lead to a better estimate. The relationship between the number of iterations and the estimate of performance depends on various factors:

Underfitting and Overfitting: If you have too few iterations, your model may underfit the training data and have poor generalization. Increasing the number of iterations can help the model fit the data better, reducing underfitting. However, if you increase iterations excessively, the model might start to overfit the training data, which can lead to poor generalization on unseen data.

Learning Rate: The learning rate, which controls the size of parameter updates during training, can interact with the number of iterations. Higher learning rates might require fewer iterations to converge, but they can also lead to instability or overshooting. Smaller learning rates might necessitate more iterations for convergence.

Early Stopping: Many training procedures use techniques like early stopping to prevent overfitting. In this case, training stops when the model's performance on a validation set starts to degrade, which means the number of iterations is dynamically determined.

Data Complexity: The complexity of your dataset and the model architecture also play a role. Complex models on complex datasets might require more iterations to reach convergence.

Computational Resources: Training for a higher number of iterations can be computationally expensive. You might need to balance computational constraints with the need for more iterations.

In practice, it's common to use techniques like cross-validation or validation sets to find an appropriate number of iterations that balances underfitting and overfitting. You typically monitor the model's performance on validation data and choose the number of iterations that yields the best generalization without overfitting.

So, while increasing the number of iterations can help improve model performance up to a point, there's no guarantee that more iterations always lead to a better estimate, as it depends on several factors, including those mentioned above. The key is to find the right balance during model training to achieve the best generalization performance.




User
Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

ChatGPT
Increasing the number of iterations can help to some extent when dealing with a very small training dataset or validation dataset, but it's not a complete solution to the challenges posed by limited data. Here's why:

Overfitting Risk: When you have a very small dataset, increasing the number of iterations during training can lead to a higher risk of overfitting. The model may start memorizing the training examples rather than learning general patterns, which can result in poor performance on unseen data.

Limited Diversity: Small datasets often lack the diversity and variety needed to capture the full range of patterns and variations in the underlying data distribution. No matter how many iterations you use, the model may struggle to generalize well because it hasn't seen enough diverse examples.

Validation Set Limitation: A small validation dataset can also be problematic because it may not provide a reliable estimate of the model's performance. Randomness in data splitting becomes more pronounced with smaller validation sets, and the estimated performance can be highly variable.

Resource Constraints: Increasing iterations can be computationally expensive, and there may be practical limitations on how many iterations you can afford to run, especially with small datasets.

To address these challenges with small datasets, consider the following strategies:

Data Augmentation: If possible, apply data augmentation techniques to artificially increase the size of your training dataset. This can help expose the model to more diverse examples.

Transfer Learning: Use pre-trained models and fine-tuning on your small dataset. Transfer learning leverages knowledge learned from larger datasets, which can be especially effective when data is limited.

Regularization: Employ regularization techniques like dropout or weight decay to mitigate overfitting when you increase the number of iterations.

Cross-Validation: Use cross-validation with your small dataset to get a more robust estimate of model performance. It helps in assessing how well your model generalizes across different subsets of the data.

Bayesian Optimization: Experiment with hyperparameter tuning methods, such as Bayesian optimization, to find the best combination of hyperparameters, including the number of iterations.

In summary, while increasing the number of iterations can have some benefits, it's important to consider the limitations of small datasets and employ a combination of techniques, including data augmentation, regularization, and cross-validation, to address the challenges associated with limited data effectively.